In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import quandl as ql

import bs4 as bs    
import pickle    
import requests    
import lxml

import os.path
import urllib
from IPython.display import clear_output

%matplotlib inline

In [3]:
#pulls the information from Wikipedia on the current S&P500 companies
def saveTickersSP500():
    tickers = []
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')        
    soup = bs.BeautifulSoup(resp.text,'lxml')        
    table = soup.find('table', {'class': 'wikitable sortable'})        

    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.split('\n')[0])
        
    clear_output()
        
    return tickers    

In [4]:
#builds the url needed to make the API call for the raw data, given a ticker symbol and api key
def makeUrl(ticker_symbol, date):
    lock = "ARgNDD47ESTvwYsQbsGM"
    base_url = "https://www.quandl.com/api/v3/datasets/WIKI/"
    return base_url + "{0}.csv?start_date={1}&api_key={2}".format(ticker_symbol,date,lock)

#Creates the output path, given the ticker symbol
def makeFilename(ticker_symbol):
    output_path = "C:/Users/joema/Desktop/Refactored_Py_DS_ML_Bootcamp-master/Stock Project/S&P"
    return output_path + "/" + ticker_symbol + ".csv"

#makes the API call for the ticker symbol and stores it in the desired output path
def pullHistoricalData(ticker_symbol, date):
    try:
        urllib.request.urlretrieve(makeUrl(ticker_symbol, date), makeFilename(ticker_symbol))
    except Exception as e:
        print("Unable to create CSV for : " + ticker_symbol)
        print(e)

In [6]:
#refreshes raw data - however should allow for passing in a user defined date
def refreshData():
    tickers = saveTickersSP500()
    for i in tickers:
        pullHistoricalData(i, "2007-01-01")

In [7]:
#returns the raw data from the local directory, and performs a reindex to present the data in reverse
#chronological order
def readTickerCSV(ticker_symbol):
        df = pd.read_csv('C:/Users/joema/Desktop/Refactored_Py_DS_ML_Bootcamp-master/Stock Project/S&P/'
                         + '{0}.csv'.format(ticker_symbol))
        df = df.reindex(index=df.index[::-1])
        df.reset_index(drop=True,inplace=True)
        return df

In [8]:
#returns the cleaned CSV from the local directory
def readCleanedTickerCSV(ticker_symbol):
        df = pd.read_csv('C:/Users/joema/Desktop/Refactored_Py_DS_ML_Bootcamp-master/Stock Project/S_P_Cleaned/'
                         + '{0}_cleaned.csv'.format(ticker_symbol))
        return df

In [9]:
#retrieves the tickers in the SP500
#Cleaned variable is to return the cleaned Dataframes stored locally
def tickerList(cleaned: bool):
    tickers = saveTickersSP500()
    ticker_list = {}
    for i in tickers:
        try:
            if cleaned:
                ticker_list[i] = readCleanedTickerCSV(i)
            else:
                ticker_list[i] = readTickerCSV(i)
        except Exception as e:
            continue
    return ticker_list

In [10]:
#remove the columns not used in the analysis
#create columns in the DF for the EMA's (3,10,30,100)
#create columns for the logic needed to invest or not
def cleanDf(df):
    df['adjClose'] = df['Adj. Close']
    df = df[['Date','adjClose']]
    df['3EMA'] = df.adjClose.ewm(span=3, adjust=False).mean()
    df['10EMA'] = df.adjClose.ewm(span=10, adjust=False).mean()
    df['30EMA'] = df.adjClose.ewm(span=30, adjust=False).mean()
    df['100EMA'] = df.adjClose.ewm(span=100, adjust=False).mean()
    df['Invest'] = 0
    df['LongTerm'] = 0
    df['Cash'] = 0
    df['Shares'] = 0
    df['RoI'] = 0
    print('cleaning done')
    return(df)

In [56]:
#In the column LongTerm, set each row to 1 if:
#the 30 EMA is greater than the 100EMA
def setBoundries(df):
    for i in range(len(df)):
        if df['30EMA'][i] > df['100EMA'][i]:
            df['LongTerm'][i] = 1
    return (df)

#in the column Invest, set each row to 1 if: 
#1 - LongTerm equals 1, and
#2 - the 3 day EMA is greater than the 10 day
def setInvest(df):
    for i in range(len(df)):
        if df['LongTerm'][i] == 1:
            if df['3EMA'][i] > df['10EMA'][i]:
                df['Invest'][i] = 1
    return(df)

In [12]:
def scoreStock(df, initial):    
    cash = initial
    initial = initial
    shares = 0

    for i in range(len(df)):
        try:
            #the logic below at a high level is to invest when the current row minus the previous row equals 1
            invest = df['Invest'][i] - df['Invest'][(i - 1)]
            if invest == 1:
                if df['LongTerm'][i] == 1:
                    #buy stock if longterm and invest = 1
                    shares = cash / df['adjClose'][i]
                    cash = cash % df['adjClose'][i]
                    
                    #update shares and cash columns after buying with a full investment
                    df['Shares'][i] = shares
                    df['Cash'][i] = cash
                else:
                    #sell the stock if Long Term switches to 0
                    cash = shares * df['adjClose'][i]
                    
                    
                    #update shares and cash columns after buying with a full investment
                    shares = 0

                    df['Shares'][i] = shares
                    df['Cash'][i] = cash
                
            #the logic below is for when the current row minus the previous row equals -1, 
            #signaling the investment should be sold
            elif invest == -1:
                cash = shares * df['adjClose'][i]
    
                shares = 0

                df['Shares'][i] = shares
                df['Cash'][i] = cash
                
            else:
                df['Shares'][i] = shares
                df['Cash'][i] = cash

        except:
                df['Shares'][i] = shares
                df['Cash'][i] = cash
    return df

In [13]:
def findRoi(df, initial):
    #calculate ROI for each row of the given dataframe, based on the intial investment
    df['RoI'] = (((df['Cash'] + (df['Shares'] * df['adjClose'])) - initial) / initial) * 100
    return df

In [ ]:
tickers = tickerList(False)

In [ ]:
for i in tickers:
    tickers[i] = cleanDf(tickers[i])
    tickers[i] = setBoundries(tickers[i], '30', '100')
    tickers[i] = setInvest(tickers[i], '3', '10')
    tickers[i] = scoreStock(tickers[i], 1000)
    tickers[i] = findRoi(tickers[i], 1000)
    print(i)